## Trabalho Prático 01 - Aprendizagem de Máquina
- Thiago Martin Poppe
- 2017014324

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.model_selection import StratifiedKFold

from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

## Lendo o dataset

- Lendo os dados e imprimindo as 5 primeiras linhas para ter uma visualização geral

In [2]:
df = pd.read_csv('koi_data.csv')
print("Dataset shape:", df.shape)
df.head()

Dataset shape: (5202, 43)


,kepoi_name,koi_disposition,koi_period,koi_impact,koi_duration,koi_depth,koi_ror,koi_srho,koi_prad,koi_sma,...,koi_fwm_srao,koi_fwm_sdeco,koi_fwm_prao,koi_fwm_pdeco,koi_dicco_mra,koi_dicco_mdec,koi_dicco_msky,koi_dikco_mra,koi_dikco_mdec,koi_dikco_msky
0,K00752.01,CONFIRMED,9.48804,0.146,2.95750,615.8,0.02234,3.20796,2.26,0.0853,...,0.430,0.940,-0.00020,-0.00055,-0.010,0.200,0.200,0.080,0.310,0.320
1,K00752.02,CONFIRMED,54.41838,0.586,4.50700,874.8,0.02795,3.02368,2.83,0.2734,...,-0.630,1.230,0.00066,-0.00105,0.390,0.000,0.390,0.490,0.120,0.500
2,K00754.01,FALSE POSITIVE,1.73695,1.276,2.40641,8079.2,0.38739,0.22080,33.46,0.0267,...,-0.111,0.002,0.00302,-0.00142,-0.249,0.147,0.289,-0.257,0.099,0.276
3,K00755.01,CONFIRMED,2.52559,0.701,1.65450,603.3,0.02406,1.98635,2.75,0.0374,...,-0.010,0.230,0.00008,-0.00007,0.030,-0.090,0.100,0.070,0.020,0.070
4,K00114.01,FALSE POSITIVE,7.36179,1.169,5.02200,233.7,0.18339,0.00485,39.21,0.0820,...,-13.450,24.090,0.00303,-0.00555,-4.506,7.710,8.930,-4.537,7.713,8.948


## Convertendo dados categóricos para numéricos
- Iremos converter CONFIRMED para 1 e FALSE POSITIVE para 0

In [3]:
df['koi_disposition'] = (df['koi_disposition'] == 'CONFIRMED').astype(int)
df.head()

,kepoi_name,koi_disposition,koi_period,koi_impact,koi_duration,koi_depth,koi_ror,koi_srho,koi_prad,koi_sma,...,koi_fwm_srao,koi_fwm_sdeco,koi_fwm_prao,koi_fwm_pdeco,koi_dicco_mra,koi_dicco_mdec,koi_dicco_msky,koi_dikco_mra,koi_dikco_mdec,koi_dikco_msky
0,K00752.01,1,9.48804,0.146,2.95750,615.8,0.02234,3.20796,2.26,0.0853,...,0.430,0.940,-0.00020,-0.00055,-0.010,0.200,0.200,0.080,0.310,0.320
1,K00752.02,1,54.41838,0.586,4.50700,874.8,0.02795,3.02368,2.83,0.2734,...,-0.630,1.230,0.00066,-0.00105,0.390,0.000,0.390,0.490,0.120,0.500
2,K00754.01,0,1.73695,1.276,2.40641,8079.2,0.38739,0.22080,33.46,0.0267,...,-0.111,0.002,0.00302,-0.00142,-0.249,0.147,0.289,-0.257,0.099,0.276
3,K00755.01,1,2.52559,0.701,1.65450,603.3,0.02406,1.98635,2.75,0.0374,...,-0.010,0.230,0.00008,-0.00007,0.030,-0.090,0.100,0.070,0.020,0.070
4,K00114.01,0,7.36179,1.169,5.02200,233.7,0.18339,0.00485,39.21,0.0820,...,-13.450,24.090,0.00303,-0.00555,-4.506,7.710,8.930,-4.537,7.713,8.948


## Separando entre classe e atributos

- Separando valores entre y (CONFIRMED ou FALSE POSITIVE) e X (features).
- Normalizando as features para que sua distribuição $\approx$ N(0, 1): $$x' = \frac{x - mean(x)}{std(x)}$$

In [4]:
y = df['koi_disposition']
X = df.drop(['kepoi_name', 'koi_disposition'], axis=1) # nome e classe não são atributos

X = (X - np.mean(X)) / np.std(X, ddof=1)
X.head()

,koi_period,koi_impact,koi_duration,koi_depth,koi_ror,koi_srho,koi_prad,koi_sma,koi_incl,koi_teq,...,koi_fwm_srao,koi_fwm_sdeco,koi_fwm_prao,koi_fwm_pdeco,koi_dicco_mra,koi_dicco_mdec,koi_dicco_msky,koi_dikco_mra,koi_dikco_mdec,koi_dikco_msky
0,-0.311523,-0.217299,-0.380535,-0.309368,-0.082308,-0.008253,-0.029723,-0.301276,0.519877,-0.452083,...,0.071564,0.118416,0.000962,-0.012755,0.016119,0.104646,-0.549713,0.048032,0.149462,-0.509178
1,0.196636,-0.049884,-0.157990,-0.305502,-0.080138,-0.015586,-0.029569,0.476664,0.514358,-0.903237,...,-0.024986,0.138089,0.014050,-0.019205,0.178346,0.031827,-0.489349,0.214354,0.079985,-0.451904
2,-0.399187,0.212652,-0.459685,-0.197957,0.058845,-0.127115,-0.021291,-0.543633,-0.864035,0.323901,...,0.022287,0.054786,0.049967,-0.023978,-0.080812,0.085349,-0.521437,-0.088677,0.072306,-0.523178
3,-0.390267,-0.006128,-0.567677,-0.309555,-0.081642,-0.056862,-0.029591,-0.499380,0.259282,0.338080,...,0.031487,0.070253,0.005223,-0.006563,0.032341,-0.000942,-0.581484,0.043975,0.043418,-0.588726
4,-0.335570,0.171940,-0.084024,-0.315072,-0.020035,-0.135708,-0.019736,-0.314924,-1.242358,0.255584,...,-1.192705,1.688816,0.050119,-0.077256,-1.807321,2.839001,2.223870,-1.824919,2.856491,2.236176


## Validação Cruzada

- Usaremos a classe StratifiedKFold, disponível em ``sklearn.model_selection``, para realizar a validação cruzada com 5-folds.
- Instanciaremos a classe para uso posterior.

In [5]:
X = X.values
y = y.values

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42) # fixed seed
print("Número de splits:", skf.get_n_splits())

Número de splits: 5


## Criação da tabela de métricas

- Usaremos essa tabela para comparar os métodos no que tange acurácia em treino e validação, recall e precision.
- Para os métodos ainda não testados usaremos NaN.

In [6]:
_models = ['Naive Bayes', 'Decision Tree', 'SVM', 'k-NN', 'Random Forest', 'Gradient Tree Boosting']
_metrics = ['Train Accuracy', 'Validation Accuracy', 'Recall', 'Precision']

df = pd.DataFrame(index=_models, columns=_metrics)
df

,Train Accuracy,Validation Accuracy,Recall,Precision
Naive Bayes,NaN,NaN,NaN,NaN
Decision Tree,NaN,NaN,NaN,NaN
SVM,NaN,NaN,NaN,NaN
k-NN,NaN,NaN,NaN,NaN
Random Forest,NaN,NaN,NaN,NaN
Gradient Tree Boosting,NaN,NaN,NaN,NaN


## Função para treinar um modelo genérico

- Função que treina um classificador, retornando um dicionário contendo acurácia do treino e validação, recall e precision.

In [7]:
def train_model(model):
    # Criando lista para cada métrica
    train_acc = []
    val_acc = []
    recall = []
    precision = []

    # Dividindo os dados em treino e validação (5 folds)
    for train, val in skf.split(X, y):
        # Treinando o modelo
        model.fit(X[train], y[train])

        # Calculando acurácia no treino
        y_pred = classifier.predict(X[train])
        train_acc.append(metrics.accuracy_score(y[train], y_pred))

        # Guardando métricas da validação
        y_pred = classifier.predict(X[val])
        val_acc.append(metrics.accuracy_score(y[val], y_pred))
        recall.append(metrics.recall_score(y[val], y_pred))
        precision.append(metrics.precision_score(y[val], y_pred))
    
    result = {
        'Train Accuracy': np.mean(train_acc),
        'Validation Accuracy': np.mean(val_acc),
        'Recall': np.mean(recall),
        'Precision': np.mean(precision)
    }
    
    return result

## 1) Naive Bayes

- Treinamento de um modelo Naive Bayes Gaussiano através da classe GaussianNB, disponível em ``sklearn.naive_bayes``.
- Esse modelo servirá de baseline para os próximos.

### 1.1) Treinamento do modelo

In [8]:
classifier = GaussianNB()

# Salvando métricas computadas e exibindo o resultado
df.loc['Naive Bayes'] = train_model(classifier)

df

,Train Accuracy,Validation Accuracy,Recall,Precision
Naive Bayes,0.91633,0.917338,0.972441,0.846197
Decision Tree,NaN,NaN,NaN,NaN
SVM,NaN,NaN,NaN,NaN
k-NN,NaN,NaN,NaN,NaN
Random Forest,NaN,NaN,NaN,NaN
Gradient Tree Boosting,NaN,NaN,NaN,NaN


### 1.2) Explicação do modelo

- O modelo Naive Bayes faz parte de uma família de classificadores simples baseados em aplicações do teorema de Bayes. Nela supomos fortemente que as features são independentes entre si, explicando assim a parte "ingênua" do algoritmo.
- A escolha do conjunto de features se torna importante, pois podemos escolher um quantia boa de features independentes entre si, melhorando assim a acurácia do nosso modelo.

- Teorema de Bayes:
$$ P(A|B) = \frac{P(B|A) * P(A)}{P(B)} $$

## 2) Árvores de Decisão

- Treinamento de um modelo Decision Tree através da classe DecisionTreeClassifier, disponível em ``sklearn.tree``.
- Para este experimento iremos variar o hiperparâmetro correspondente à altura da árvore.

### 2.1) Treinamento do modelo

In [ ]:
param_metrics = dict()

# Treinando modelo para alturas de 1 até 20
for h in range(1, 21):
    classifier = DecisionTreeClassifier(max_depth=h)
    param_metrics[h] = train_model(classifier)
    
# Treinando modelo para altura ilimitada
classifier = DecisionTreeClassifier(max_depth=None)
param_metrics['Inf'] = train_model(classifier)

### 2.2) Resultado gráfico do experimento

In [ ]:
x = param_metrics.keys()
train_acc = [metric['Train Accuracy']      for metric in param_metrics.values()]
val_acc   = [metric['Validation Accuracy'] for metric in param_metrics.values()]

# Definindo labels do plot
plt.title("Decision Tree Model")
plt.ylabel("Accuracy")
plt.xlabel("Heights")

# Plotando acurácia no treino e validação
plt.plot(x, train_acc, label='Train')
plt.plot(x, val_acc, label='Validation')

plt.legend()
plt.show()

### 2.3) Guardando modelo que apresentou os melhores resultados

In [ ]:
classifier = DecisionTreeClassifier(max_depth=5)
df.loc['Decision Tree'] = train_model(classifier)
df

- Para esse experimento variamos a altura da árvore de decisão entre os valores 1 até 20, incluindo no final uma altura ilimitada. Para modificarmos esse hiperparâmetro, utilizamos o parâmetro ``max_depth`` presente na classe ``DecisionTreeClassifier``, onde ``max_depth=None`` representa uma árvore de decisão com altura ilimitada, isto é, uma árvore que possui todas as folhas puras ou até que não se possa fazer mais splits. <br><br>
- Como observado no gráfico da seção 2.2, temos que uma árvore com altura ilimitada leva à uma acurácia de 100% no treino. Por mais interessante que esse valor seja estamos com uma diferença muito grande entre as curvas de treino e validação, indicando assim um overfit dos dados. <br><br>
- Teremos que o melhor modelo para os dados escolhidos será uma árvore de altura igual à 5, pois após esse valor não temos um aumento da acurácia significativo na validação e as curvas apenas se distanciam.

## 3) SVM (Support Vector Machine)

- Treinamento de um modelo SVM (Support Vector Machine) através da classe SVC (Support Vector Classification), disponível em ``sklearn.svm``.
- Para este experimento iremos variar o hiperparâmetro correspondente ao kernel utilizado pelo SVM, podendo ser Linear, Sigmoid, Polinomial e RBF (radial basis function).

### 3.1) Treinamento do modelo

In [ ]:
param_metrics = dict()
kernels = ['linear', 'sigmoid', 'poly', 'rbf']

# Treinando modelo para alturas de 1 até 20
for k in kernels:
    classifier = SVC(kernel=k)
    param_metrics[k] = train_model(classifier)

### 3.2) Resultado do experimento

In [ ]:
cols = ['Train Accuracy', 'Validation Accuracy', 'Recall', 'Precision']

table = pd.DataFrame(param_metrics.values(), index=kernels, columns=cols)
table

### 3.3) Guardando modelo que apresentou os melhores resultados

In [ ]:
classifier = SVC(kernel='linear')
df.loc['SVM'] = train_model(classifier)
df

- Para esse experimento variamos a altura da árvore de decisão entre os valores 1 até 20, incluindo no final uma altura ilimitada. Para modificarmos esse hiperparâmetro, utilizamos o parâmetro ``max_depth`` presente na classe ``DecisionTreeClassifier``, onde ``max_depth=None`` representa uma árvore de decisão com altura ilimitada, isto é, uma árvore que possui todas as folhas puras ou até que não se possa fazer mais splits. <br><br>
- Como observado no gráfico da seção 2.2, temos que uma árvore com altura ilimitada leva à uma acurácia de 100% no treino. Por mais interessante que esse valor seja estamos com uma diferença muito grande entre as curvas de treino e validação, indicando assim um overfit dos dados. <br><br>
- Teremos que o melhor modelo para os dados escolhidos será uma árvore de altura igual à 5, pois após esse valor não temos um aumento da acurácia significativo na validação e as curvas apenas se distanciam.